In [ ]:
# This is a demo notebook for implementing csr sp*sp products in Python
# there are no cool gradient things happening in here

In [ ]:
import numpy as np
import scipy.sparse as sp

In [ ]:
A = sp.eye(5, k=-1).tocsr()
B = sp.eye(5, k=-1).tocsr()

In [ ]:
(A.T@B).todense()

In [ ]:
(A@B.T).todense()

In [ ]:
(A@B).todense()

In [ ]:
def spmmAtB(A_shape, A_data, A_indptr, A_indices,
            B_shape, B_data, B_indptr, B_indices):
    C = np.zeros((A_shape[1], B_shape[1]))
    
    for A_row in range(len(A_indptr)-1):
        for A_row_i in range(A_indptr[A_row], A_indptr[A_row+1]):
            A_col = A_indices[A_row_i]
            for B_row_i in range(B_indptr[A_row], B_indptr[A_row+1]):
                B_col = B_indices[B_row_i]
                    
                C[A_col, B_col] += A_data[A_row_i] * B_data[B_row_i]
    
    return C

In [ ]:
def spmmABt(A_shape, A_data, A_indptr, A_indices,
            B_shape, B_data, B_indptr, B_indices):
    C = np.zeros((A_shape[0], B_shape[0]))
    
    for A_row in range(len(A_indptr)-1):
        for B_row in range(len(B_indptr)-1):
            A_row_i = A_indptr[A_row]
            B_row_i = B_indptr[B_row]
            
            A_row_end = A_indptr[A_row + 1]
            B_row_end = B_indptr[B_row + 1]
            
            cij = 0.
            
            while A_row_i < A_row_end and B_row_i < B_row_end:
                A_col = A_indices[A_row_i]
                B_col = B_indices[B_row_i]
                
                if A_col < B_col:
                    A_row_i += 1
                elif A_col > B_col:
                    B_row_i += 1
                else:
                    cij += A_data[A_row_i] * B_data[B_row_i]
                    A_row_i += 1
                    B_row_i += 1

            C[A_row][B_row] = cij
    
    return C

In [ ]:
def spmmAB(A_shape, A_data, A_indptr, A_indices,
            B_shape, B_data, B_indptr, B_indices):
    C = np.zeros((A_shape[1], B_shape[1]))
    
    for A_row in range(len(A_indptr)-1):
        for A_row_i in range(A_indptr[A_row], A_indptr[A_row+1]):
            A_col = A_indices[A_row_i]
            for B_row_i in range(B_indptr[A_col], B_indptr[A_col+1]):
                B_col = B_indices[B_row_i]
                    
                C[A_row, B_col] += A_data[A_row_i] * B_data[B_row_i]
    
    return C

In [ ]:
spmmAtB(A.shape, A.data, A.indptr, A.indices,
        B.shape, B.data, B.indptr, B.indices)

In [ ]:
spmmABt(A.shape, A.data, A.indptr, A.indices,
        B.shape, B.data, B.indptr, B.indices)

In [ ]:
spmmAB(A.shape, A.data, A.indptr, A.indices,
       B.shape, B.data, B.indptr, B.indices)